Topic: Project 5  
Subject: Work Pipeline for Building Playlist  
Date: 12/12/2017  
Name: Zach Heick  

**Summary**: Use the Song2vec model and other features to put songs into feature space for recommending

In [1]:
import boto3
import random
import os
import pandas as pd
import pickle
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
from gensim.models import doc2vec
from collections import namedtuple
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Download Model and Lyrics Dataframe from S3

In [2]:
s3 = boto3.client('s3', 
                  aws_access_key_id=os.environ['AWS_ACCESS_KEY'], 
                  aws_secret_access_key=os.environ['AWS_SECRET_KEY'])

In [3]:
s3.download_file('metis-project-kojak-bucket', 'all_lyrics_dataframe.pickle', '../Data_Files/all_lyrics_dataframe.pickle')

In [4]:
s3.download_file('metis-project-kojak-bucket', 'song2vec_model_v2', '../Data_Files/song2vec_model_v2')

In [5]:
with open('../Data_Files/song2vec_model_v2', 'rb') as f:
    model = pickle.load(f)

In [6]:
with open('../Data_Files/all_lyrics_dataframe.pickle', 'rb') as f:
    df = pickle.load(f)

In [7]:
len(df)

19831

# Get Weights Matrix

In [8]:
weights_df = pd.DataFrame(data=list(model.docvecs), columns=['w'+str(i) for i in range(100)])
weights_df.head()

,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,...,w90,w91,w92,w93,w94,w95,w96,w97,w98,w99
0,-2.400560,4.507967,-2.713643,0.157224,0.637972,-1.686617,0.947925,1.663428,-4.311983,2.468158,...,-1.033367,1.916137,0.764904,-1.631729,0.024065,0.249286,-1.606547,-0.850924,3.502970,2.007836
1,-1.493115,3.556296,-6.074947,-1.316360,5.679130,3.472410,4.320534,3.758375,0.277765,2.650105,...,-0.732355,0.636218,-0.920253,-6.260690,-4.694861,1.657172,-4.711178,-1.381411,1.318161,-2.139941
2,-0.486226,2.345176,-0.106771,-1.927061,0.547968,1.693201,0.360235,-0.403875,-1.410558,-0.324914,...,0.352682,-1.016584,-0.255911,-0.815822,-0.252194,-0.858311,0.148211,-0.173675,-0.548015,0.659603
3,3.148485,1.718158,-0.447945,2.209822,-2.385956,3.026508,0.013219,1.402475,-1.951970,1.953212,...,3.770243,0.620282,0.669534,-4.667129,-0.999159,-2.142076,-3.167053,-3.528321,1.120637,-0.457479
4,2.820393,2.929027,-4.379201,-2.696359,4.695037,-0.163835,-0.069367,-1.647813,-0.326839,-0.478175,...,1.016936,-1.417165,3.015800,-1.405270,1.155383,0.721039,0.167504,1.780159,-2.046462,-3.944089


# Dimensionality Reduction

In [9]:
components = 25

In [10]:
svd = TruncatedSVD(n_components=components, n_iter=10, random_state=1234)
weights_svd = svd.fit_transform(weights_df) 

In [11]:
weights_svd_df = pd.DataFrame(data=weights_svd, columns=['x'+str(i) for i in range(components)])
weights_svd_df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24
0,7.674929,-3.117137,-5.316305,-1.563437,-3.071741,2.414253,-0.133019,-4.597830,-3.243381,0.212409,...,-1.651078,0.560601,3.262218,0.244277,4.948557,-1.330755,0.437911,1.767593,0.880353,1.378531
1,9.110387,-0.953863,-1.570719,-2.914395,5.280568,-5.648518,1.911740,0.543203,-6.417181,0.296061,...,3.501916,3.335733,0.987961,0.523496,0.218431,5.814930,-1.944734,7.772025,-0.005365,-0.090708
2,6.157136,1.607168,-1.834804,0.218988,-1.823901,0.787692,1.202934,0.337620,-0.497898,-0.139979,...,-2.142735,-0.266476,-2.243062,1.362036,-0.855733,-0.421496,0.018040,-0.343307,0.407784,1.937628
3,12.712217,1.320403,0.009973,6.159326,0.746395,-0.210556,-2.602195,1.574887,1.971461,2.313310,...,-2.783311,3.616478,2.432844,0.468928,-2.489764,0.447557,-0.586870,-2.627744,0.366989,3.269173
4,8.363818,4.551073,4.097313,-3.734912,-0.604490,-2.364369,-1.058493,1.457608,-3.830490,1.570972,...,-1.814827,-4.432541,-1.915347,-0.839718,-0.770473,-0.856591,-2.754213,0.614055,-1.003948,1.153068


# Other Features

In [12]:
df.columns

Index(['album_art', 'album_name', 'duration_ms', 'energy', 'liveness',
       'loudness', 'speechiness', 'tempo', 'track_id', 'track_name',
       'artist_popularity', 'artist_name', 'artist_id', 'lyrics'],
      dtype='object')

In [13]:
track_metadata = df[['energy', 'speechiness', 'tempo']]
track_metadata.head()

,energy,speechiness,tempo
0,0.809,0.566,146.216
1,0.649,0.455,127.635
2,0.813,0.226,102.926
3,0.665,0.146,138.036
4,0.586,0.216,89.998


In [14]:
ss = StandardScaler()
track_metadata_ss = pd.DataFrame(data=ss.fit_transform(track_metadata), columns=track_metadata.columns)

In [15]:
tracks = pd.concat([weights_svd_df, track_metadata_ss], axis=1)
tracks.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x18,x19,x20,x21,x22,x23,x24,energy,speechiness,tempo
0,7.674929,-3.117137,-5.316305,-1.563437,-3.071741,2.414253,-0.133019,-4.597830,-3.243381,0.212409,...,0.244277,4.948557,-1.330755,0.437911,1.767593,0.880353,1.378531,0.670076,2.044680,1.012479
1,9.110387,-0.953863,-1.570719,-2.914395,5.280568,-5.648518,1.911740,0.543203,-6.417181,0.296061,...,0.523496,0.218431,5.814930,-1.944734,7.772025,-0.005365,-0.090708,-0.314871,1.331945,0.433140
2,6.157136,1.607168,-1.834804,0.218988,-1.823901,0.787692,1.202934,0.337620,-0.497898,-0.139979,...,1.362036,-0.855733,-0.421496,0.018040,-0.343307,0.407784,1.937628,0.694700,-0.138473,-0.337266
3,12.712217,1.320403,0.009973,6.159326,0.746395,-0.210556,-2.602195,1.574887,1.971461,2.313310,...,0.468928,-2.489764,0.447557,-0.586870,-2.627744,0.366989,3.269173,-0.216376,-0.652155,0.757434
4,8.363818,4.551073,4.097313,-3.734912,-0.604490,-2.364369,-1.058493,1.457608,-3.830490,1.570972,...,-0.839718,-0.770473,-0.856591,-2.754213,0.614055,-1.003948,1.153068,-0.702694,-0.202683,-0.740349


# Nearest Neighbors

In [16]:
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(tracks)

In [17]:
distances, indices = nn.kneighbors(tracks)

In [18]:
nn_d = {}
for track_idx, neighbors in enumerate(indices):
    nn_d[track_idx] = neighbors

# Get Similar Tracks

In [19]:
def get_similar_tracks(artist_index):
    test = nn_d[artist_index]
    print(test, '\n')
    for track in df.iloc[test].iterrows():
        artist_name = track[1]['artist_name']
        track_name = track[1]['track_name']
        lyrics = track[1]['lyrics']
        print(track_name)
        print(artist_name)
        print('===============\n')
        print(lyrics)
        print('\n--------------------------\n')

In [20]:
rand_artist = random.randint(0,19917)
print(rand_artist)
get_similar_tracks(13674)

3462
[13674  1826 11449 16706 19267 10219  9940  8312 11874 12312] 

Real Friends
Kanye West

Real friends, how many of us?
How many of us, how many jealous? Real friends
It's not many of us, we smile at each other
But how many honest? Trust issues
Switched up the number, I can't be bothered
I cannot blame you for havin' an angle
I ain't got no issues, I'm just doin' my thing
Hope you're doin' your thing too
I'm a deadbeat cousin, I hate family reunions
Fuck the church up by drinkin' after communion
Spillin' free wine, now my tux is ruined
In town for a day, what the fuck we doin'?
Who your real friends? We all came from the bottom
I'm always blamin' you, but what's sad, you not the problem
Damn I forgot to call her, shit I thought it was Thursday
Why you wait a week to call my phone in the first place?
When was the last time I remembered a birthday?
When was the last time I wasn't in a hurry?

Tell me you want your tickets when it's gametime
Even to call your daughter on her FaceTime


# Compare to Doc2vec without Mood Features

In [21]:
def get_similar_tracks_2(artist_index):
    sims = model.docvecs.most_similar(artist_index, topn=model.docvecs.count)
    sim_ids = [pair[0] for pair in sims[0:10]]
    
    main_track = df.iloc[artist_index]
    print(main_track['artist_name'])
    print(main_track['track_name'])
    print('===============\n')
    print(main_track['lyrics'])
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n')
    
    for i, track in enumerate(df.iloc[sim_ids].iterrows()):
        artist_name = track[1]['artist_name']
        track_name = track[1]['track_name']
        lyrics = track[1]['lyrics']
        print(track_name)
        print(artist_name)
        print(sims[i][1])
        print('===============\n')
        print(lyrics)
        print('\n--------------------------\n')

In [22]:
get_similar_tracks_2(13674)

Kanye West
Real Friends

Real friends, how many of us?
How many of us, how many jealous? Real friends
It's not many of us, we smile at each other
But how many honest? Trust issues
Switched up the number, I can't be bothered
I cannot blame you for havin' an angle
I ain't got no issues, I'm just doin' my thing
Hope you're doin' your thing too
I'm a deadbeat cousin, I hate family reunions
Fuck the church up by drinkin' after communion
Spillin' free wine, now my tux is ruined
In town for a day, what the fuck we doin'?
Who your real friends? We all came from the bottom
I'm always blamin' you, but what's sad, you not the problem
Damn I forgot to call her, shit I thought it was Thursday
Why you wait a week to call my phone in the first place?
When was the last time I remembered a birthday?
When was the last time I wasn't in a hurry?

Tell me you want your tickets when it's gametime
Even to call your daughter on her FaceTime
Even when we was young I used to make time
Now we be way too busy jus

In [23]:
with open('../Data_Files/neighbors', 'wb') as f:
    pickle.dump(nn_d, f)